I've not really played around with Tabular data before, so many things are missing and this is largely a regurgitation of the official fastai docs (https://docs.fast.ai/tutorial.tabular.html). 

Need to add some things such as incorporaing the dates (next up), transcation/oil/holiday data needs to be added as well, but V0 confirms pipeline can be used and output submitted to leaderboard

This can be run on a CPU, but if you invoke the GPU, it will be certainly be faster per epoch

In [ ]:
from fastai.tabular.all import *
from sklearn.metrics import mean_squared_log_error

In [ ]:
path = Path('../input/store-sales-time-series-forecasting')
Path.BASE_PATH = path
path.ls()

In [ ]:
train_df = pd.read_csv(path/'train.csv')
transaction_df = pd.read_csv(path/'transactions.csv')
test_df = pd.read_csv(path/'test.csv')
sub_df = pd.read_csv(path/'sample_submission.csv')
train_df.shape, transaction_df.shape, test_df.shape, sub_df.shape

In [ ]:
train_df.head(3)

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [ ]:
tab_pd = TabularPandas(train_df, 
                       procs = [Categorify, FillMissing, Normalize],
                       cat_names=['store_nbr', 'family'],
                       cont_names =['onpromotion'],
                       y_names = 'sales',
                       splits = splits)

In [ ]:
dls = tab_pd.dataloaders(bs=512)

In [ ]:
len(dls.train), len(dls.valid)

In [ ]:
dls.show_batch()

In [ ]:
train_df.sales.max()

In [ ]:
def rmsle(inp, targ):
    inp = inp.detach().cpu().numpy()
    targ = targ.detach().cpu().numpy()
    return np.sqrt(mean_squared_log_error(targ, inp))

In [ ]:
learner = tabular_learner(dls, metrics=rmsle, y_range=(0, 125000))

In [ ]:
learner.fit_one_cycle(1)

In [ ]:
dl = learner.dls.test_dl(test_df)

In [ ]:
preds = learner.get_preds(dl=dl)
preds[0].shape

In [ ]:
sub_df.sales = preds[0].detach().cpu().numpy()
sub_df.head(3)

In [ ]:
sub_df.to_csv('submission.csv', index=False)